# Design Patterns

**Creational Patterns**: Patterns that are used to generate objects with specific behaviors  
**Structural Patterns**: Patterns that help in structuring the code for specific use cases  
**Behavioral Patterns**: Patterns that help in structuring processes

Benefits of using design pattern are:  
 - Patterns provide developer a selection of tried and tested solutions for the specified problems.  
 - All design patterns are language neutral.  
 - Patterns help to achieve communication and maintain well documentation.  
 - It includes a record of accomplishment to reduce any technical risk to the project.  
 - Design patterns are highly flexible to use and easy to understand.  

## Creational Pattern
A creational pattern provides a particular instantiation mechanism. It can be a particular object factory or even a class factory

### Singleton
The [Singleton pattern](https://python-3-patterns-idioms-test.readthedocs.io/en/latest/Singleton.html) makes sure that a given class has always only one living instance in the application:
 - When you want to restrict a resource access to one and only one memory context in the process - It makes the assumption that no other instance is interacting with the database in the meantime
   + database connector class: synchornization and manages its data in memory
 - Utilities that provide application-wide functions are often decleared as Singletons
   + Simplify concurrency is handled in an application
   
***Implement philosophy: Delegate to a single instance of a private nested inner class***  
Some Resources:  
https://www.oodesign.com/singleton-pattern.html  
https://www.quora.com/What-is-Singleton-class

Examples:
 - **Logger Classes**: provide a global logging access point in all the application components without being necessary to create an object each time a logging operations performed
 - **Configuration Classes**: keep the instance as cache object -> when class is instantiated (or read) the Singleton will keep the values in its internal structure => avoid reloading the values each time the configuration parameters are used (if values are read from database or from files)
 - **Accessing resources in shared mode**: there are many classes in application, working in multi-threading environment, operate actions on the serial port. Singleton with synchronized methods could be used to manage all the operations
 - **Factories implemented as Singleton**: Combining Abstract Factory or Factory Method and Singleton design patterns is a common practice to avoid 2 overlapping ids for 2 different objects

In [13]:
class Singleton(object):
    # The instances of Singleton are distinct but they all proxy to the same __Singleton object.
    # The inner class contains all the methods that you would normally put in the class
    class __Singleton: 
        def __init__(self):
            self.val = None
        def __str__(self):
            return repr(self) + self.val
    
    instance = None
    
    def __new__(cls): # __new__ always a classmethod
        if not Singleton.instance:
            Singleton.instance = Singleton.__Singleton()
        return Singleton.instance
    def __getattr__(self, name):
        return getattr(self.instance, name)
    def __setattr__(self, name):
        return setattr(self.instance, name)

<__main__.Singleton.__Singleton object at 0x0000019231E45B70>sausage
<__main__.Singleton.__Singleton object at 0x0000019231E45B70>eggs
<__main__.Singleton.__Singleton object at 0x0000019231E45B70>spam
<__main__.Singleton.__Singleton object at 0x0000019231E45B70>spam
<__main__.Singleton.__Singleton object at 0x0000019231E45B70>spam


In [14]:
x = Singleton()
x.val = 'sausage'
print(x)
y = Singleton()
y.val = 'eggs'
print(y)
z = Singleton()
z.val = 'spam'
print(z)
print(x)
print(y)
print(repr(x))
print(repr(y))
print(repr(z))

<__main__.Singleton.__Singleton object at 0x0000019231E45B70>sausage
<__main__.Singleton.__Singleton object at 0x0000019231E45B70>eggs
<__main__.Singleton.__Singleton object at 0x0000019231E45B70>spam
<__main__.Singleton.__Singleton object at 0x0000019231E45B70>spam
<__main__.Singleton.__Singleton object at 0x0000019231E45B70>spam


### Borg Singleton
An implementation when we want a Singleton to have a single set of state data for all object

In [15]:
class Borg:
    _shared_state = {}
    def __init__(self):
        self.__dict__ = self._shared_state

class Singleton(Borg):
    def __init__(self, arg):
        Borg.__init__(self)
        self.val = arg
    def __str__(self): 
        return self.val

In [16]:
x = Singleton('sausage')
print(x)
y = Singleton('eggs')
print(y)
z = Singleton('spam')
print(z)
print(x)
print(y)

sausage
eggs
spam
spam
spam


In [19]:
# ClassVariableSingleton implementation
class Singleton(object):
    __instance = None
    def __new__(cls, val):
        if Singleton.__instance is None:
            Singleton.__instance = object.__new__(cls)
        Singleton.__instance.val = val
        return Singleton.__instance

### ClassVariableSingleton implementation
There's only one instance of a class variable

In [20]:
## implementation 1: Decorator
class SingletonDecorator:
    def __init__(self, klass):
        self.klass = klass
        self.instance = None
    def __call__(self, *args, **kwds):
        if self.instance ==None:
            self.instance = self.klass(*args, **kwds)
        return self.instance
    
class foo:
    pass

foo = SingletonDecorator(foo)

In [18]:
x=foo()
y=foo()
z=foo()
x.val = 'sausage'
y.val = 'eggs'
z.val = 'spam'
print(x.val)
print(y.val)
print(z.val)
print(x is y is z)

spam
spam
spam
True


In [21]:
## implementation 2: SingletonMetaClass
class SingletonMetaClass(type):
    def __init__(cls,name,bases,dict):
        super(SingletonMetaClass,cls)\
          .__init__(name,bases,dict)
        original_new = cls.__new__
        def my_new(cls,*args,**kwds):
            if cls.instance == None:
                cls.instance = \
                  original_new(cls,*args,**kwds)
            return cls.instance
        cls.instance = None
        cls.__new__ = staticmethod(my_new)

class bar(object):
    __metaclass__ = SingletonMetaClass
    def __init__(self,val):
        self.val = val
    def __str__(self):
        return repr(self) + self.val

In [22]:
x=bar('sausage')
y=bar('eggs')
z=bar('spam')
print(x)
print(y)
print(z)
print(x is y is z)

<__main__.bar object at 0x0000019231E45C50>sausage
<__main__.bar object at 0x0000019231E45C18>eggs
<__main__.bar object at 0x0000019231E45C88>spam
False
